In [38]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [39]:
# item recognition - cued recall
with open("../Data/simuS1_group3_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 100]
# df_test = df_test.loc[df_test.session < 100]

In [40]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session,pair_idx
0,186,279,BUFFALO,CHAMPION,0,0
1,255,776,CASKET,KIDNEY,0,1
2,938,1259,NAVIGATOR,SEA,0,2
3,582,1115,FLORIDA,POTATO,0,3
4,224,694,CAMERA,HAYSTACK,0,4
...,...,...,...,...,...,...
47995,327,316,CLOTHES,CLAMP,999,43
47996,812,1086,LEOPARD,PLATE,999,44
47997,967,344,OATMEAL,COLESLAW,999,45
47998,467,1048,DINER,PERCH,999,46


In [41]:
df_test

,test_itemno1,test_itemno2,test_item1,correct_ans,order,pair_idx,test,session
0,691,-1,HATCHET,1,0,14,1,0
1,924,-1,MOUNTAIN,1,0,39,1,0
2,1392,-1,STREAM,1,0,19,1,0
3,1365,-1,STAFF,0,-1,-1,1,0
4,760,-1,JOURNAL,0,-1,-1,1,0
...,...,...,...,...,...,...,...,...
119995,34,-1,APPLICATION,83,0,0,2,999
119996,1308,-1,SILVERWARE,1272,0,16,2,999
119997,923,-1,MOTORCYCLE,804,0,23,2,999
119998,1133,-1,PROTRACTOR,1412,1,29,2,999


### Run CMR-IA

In [42]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [43]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_cue = 0.5,
    beta_rec = 0,
    beta_rec_new = 0.1, # beta for retrieved item
    beta_rec_post = 0.1,
    gamma_fc = 0.5,
    gamma_cf = 0.5,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.3,
    c_thresh_ass = 0.70,
    a = 2800,
    b = 2.14,
    m = 0,
    n = 1,
    kappa = 0.02,
    lamb = 0,
    eta = 0,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 136,
    d_ass = 1,
    learn_while_retrieving = True,
    omega = 2,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.5,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.1,
 'beta_distract': None,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0,
 'omega': 2,
 'alpha': 1,
 'c_thresh': 0.3,
 'c_thresh_ass': 0.7,
 'd_ass': 1,
 'lamb': 0,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 136,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 2.14,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'gamma_fc': 0.5,
 'gamma_cf': 0.5}

In [44]:
df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params,df_study, df_test, s_mat)
df_simu['test'] = df_test['test']
df_simu

CMR2 Time: 98.8239860534668


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test
0,0,691,-1,1.0,2794.215576,0.300966,1
1,0,924,-1,1.0,2144.518555,0.424628,1
2,0,1392,-1,1.0,2308.074463,0.390283,1
3,0,1365,-1,0.0,1642.338135,0.050702,1
4,0,760,-1,0.0,1735.964111,0.076609,1
...,...,...,...,...,...,...,...
119995,999,34,-1,83.0,1640.000000,0.596242,2
119996,999,1308,-1,1272.0,1350.000000,0.822514,2
119997,999,923,-1,804.0,1460.000000,0.768052,2
119998,999,1133,-1,1412.0,1500.000000,0.753573,2


In [45]:
df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,correct
0,0,691,-1,1.0,2794.215576,0.300966,1,HATCHET,1,0,14,True
1,0,924,-1,1.0,2144.518555,0.424628,1,MOUNTAIN,1,0,39,True
2,0,1392,-1,1.0,2308.074463,0.390283,1,STREAM,1,0,19,True
3,0,1365,-1,0.0,1642.338135,0.050702,1,STAFF,0,-1,-1,True
4,0,760,-1,0.0,1735.964111,0.076609,1,JOURNAL,0,-1,-1,True
...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,34,-1,83.0,1640.000000,0.596242,2,APPLICATION,83,0,0,True
119996,999,1308,-1,1272.0,1350.000000,0.822514,2,SILVERWARE,1272,0,16,True
119997,999,923,-1,804.0,1460.000000,0.768052,2,MOTORCYCLE,804,0,23,True
119998,999,1133,-1,1412.0,1500.000000,0.753573,2,PROTRACTOR,1412,1,29,True


In [46]:
# get f_in
sessions = np.unique(df_simu.session)
tmp_corr_fin = []
tmp_omax_fin = []
test1_num = 80

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    df_tmp2 = df_test.loc[df_test.session == sess]
    tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
    tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
    tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
    tmp = np.unique(tmp)  # sort

    tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
    corrid = np.searchsorted(tmp, tmp_corr)

    corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
    omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

    tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
    tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

df_simu['corr_fin'] = tmp_corr_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,correct_ans,order,pair_idx,correct,corr_fin,omax_fin
0,0,691,-1,1.0,2794.215576,0.300966,1,HATCHET,1,0,14,True,-1.000000,-1.000000
1,0,924,-1,1.0,2144.518555,0.424628,1,MOUNTAIN,1,0,39,True,-1.000000,-1.000000
2,0,1392,-1,1.0,2308.074463,0.390283,1,STREAM,1,0,19,True,-1.000000,-1.000000
3,0,1365,-1,0.0,1642.338135,0.050702,1,STAFF,0,-1,-1,True,-1.000000,-1.000000
4,0,760,-1,0.0,1735.964111,0.076609,1,JOURNAL,0,-1,-1,True,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119995,999,34,-1,83.0,1640.000000,0.596242,2,APPLICATION,83,0,0,True,-0.379267,-0.527923
119996,999,1308,-1,1272.0,1350.000000,0.822514,2,SILVERWARE,1272,0,16,True,-0.246324,-0.644189
119997,999,923,-1,804.0,1460.000000,0.768052,2,MOTORCYCLE,804,0,23,True,-0.300570,-0.621480
119998,999,1133,-1,1412.0,1500.000000,0.753573,2,PROTRACTOR,1412,1,29,True,-0.319542,-0.507613


In [47]:
# recognition performance
df_recog = df_simu.query("test==1")
df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")

,Yes rate
correct_ans,
0,0.00000
1,0.78785


In [48]:
# cued recall performance
df_cr = df_simu.query("test==2")
df_cr.correct.mean()

0.999375

In [49]:
def get_pair(df_tmp):
    df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
    df_tmp_pair.columns = ["test1","test2"]
    df_tmp_pair.reset_index(inplace=True)
    return df_tmp_pair

df_simu_p = df_simu.query("pair_idx >= 0")
df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
df_pair

,session,level_1,pair_idx,test1,test2
0,0,0,0,0.0,1.0
1,0,1,1,0.0,1.0
2,0,2,2,1.0,1.0
3,0,3,3,0.0,1.0
4,0,4,4,0.0,1.0
...,...,...,...,...,...
39995,999,35,35,1.0,1.0
39996,999,36,36,1.0,1.0
39997,999,37,37,0.0,1.0
39998,999,38,38,1.0,1.0


In [50]:
df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all')
df_tab

test1,0.0,1.0
test2,,
0.0,0.00020,0.000425
1.0,0.21195,0.787425


In [51]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)
Yule_Q(df_tab[1][1],df_tab[0][1],df_tab[1][0],df_tab[0][0])

0.2722783525455565

In [52]:
# add 0.5 to each cell
Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)

0.28762120166673594